In [5]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False

# conclusion: only Neg, work well with constant source boundary, constant initial condition, Andrew's three Diffusivities

In [6]:
# Reference parameter - dimensional 
D_e     = 3e-10;    # Li+ diffusivity in m2/s
D_EC    = 5e-10;    # EC  diffusivity in m2/s
D_cross = -1.5e-9;  # Cross diffusivity in m2/s
L_n = 1; 
def RunAdd_D(D_e,D_EC,D_cross,L_n,t_scale):
        model = pybamm.BaseModel()
        c_e_n  = pybamm.Variable("Li+ concentration in negative electrode", domain="negative electrode")
        c_EC_n = pybamm.Variable("EC concentration in negative electrode", domain="negative electrode")

        model.variables = {"Li+ concentration in negative electrode":c_e_n,
        "EC concentration in negative electrode":c_EC_n,}
        model.length_scales = {"negative electrode":L_n}
        model.timescale = pybamm.Scalar(t_scale) 
        model.rhs = {
                c_e_n: 
                D_e*pybamm.div(pybamm.grad(c_e_n))
                + D_cross*pybamm.div(pybamm.grad(c_EC_n)),
                #+ a_n*(1-t_0plus)*j_tot_n
                #- a_n*(j_SEI_n*V_bar_EC+j_tot_n*V_bar_Li-0.5*j_SEI_n*V_bar_SEI)*c_init/F ,

                c_EC_n:
                D_cross*pybamm.div(pybamm.grad(c_e_n))
                + D_EC*pybamm.div(pybamm.grad(c_EC_n)),
                #+ (
                #    a_n*trans_i*j_tot_n/F
                #    +a_n* (1+trans_i)*j_SEI_n/F
                #    -a_n*(j_SEI_n*V_bar_EC+j_tot_n*V_bar_Li-0.5*j_SEI_n*V_bar_SEI)*c_EC_init/F
                #)
        } 
        model.boundary_conditions = {
        c_e_n: {
                "left": (pybamm.Scalar(4.5e-11), "Neumann"),
                "right": (pybamm.Scalar(4.5e-11), "Neumann")},  
        c_EC_n:{
                "left": (pybamm.Scalar(1.4e5), "Neumann"),
                "right": (pybamm.Scalar(1.4e5), "Neumann")} }
        # define geometry
        x_n = pybamm.SpatialVariable(
        "x_n", 
        domain=["negative electrode"], 
        coord_sys="cartesian"
        )
        model.initial_conditions = {
                c_e_n: pybamm.Scalar(1000),
                c_EC_n: pybamm.Scalar(4000)}
        geometry = {
        "negative electrode": {x_n: {"min": pybamm.Scalar(0), "max": pybamm.Scalar(L_n)}},
        }
        # mesh and discretise ,"separator","positive electrode"
        submesh_types = {
        "negative electrode": pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
        }
        var_pts = {x_n: 100}
        mesh = pybamm.Mesh(geometry, submesh_types, var_pts)
        spatial_methods = {
        "negative electrode": pybamm.FiniteVolume(),
        }
        disc = pybamm.Discretisation(mesh, spatial_methods)
        disc.process_model(model);

        # solve
        solver = pybamm.ScipySolver()
        #t = np.linspace(0, 1e10, 100)
        solution = solver.solve(model, [0,t_scale*10])
        pybamm.dynamic_plot(solution,[
                "Li+ concentration in negative electrode",
                "EC concentration in negative electrode",],)
        return

In [7]:
D_e     = 3e-10;    # Li+ diffusivity in m2/s
D_EC    = 5e-10;    # EC  diffusivity in m2/s
D_cross = -1.5e-9;  # Cross diffusivity in m2/s 
D=1E-10
L_n=8e-3; t_scale = L_n**2/D
RunAdd_D(D_e,D_EC,D_cross,L_n,t_scale)

interactive(children=(FloatSlider(value=0.0, description='t', max=1777.7777777777778, step=17.77777777777778),…

In [8]:
print(t_scale*100/3600)

17777.777777777777
